In [39]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [60]:
# Hyperparameters
block_size = 8
batch_size = 4
max_iters = 1000
lr = 3e-4
eval_iters = 250

In [36]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [29]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100]) 

tensor([28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36, 11,
         1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32, 39,
        42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,
         1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1, 39,
        50,  9,  1, 39, 50, 37, 25,  1, 39, 30])


In [47]:
n = int(0.8 * len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    xi = torch.randint(len(data) - block_size, (batch_size,)) # random starting indices
    # print(f"Training Integers: {xi}")
    # input batch
    x = torch.stack([data[i:i+block_size] for i in xi])
    # target batch    
    y = torch.stack([data[i+1:i+1+block_size] for i in xi])
    x, y = x.to(device), y.to(device) # send to cuda -> parallelize
    return x, y

x, y = get_batch('train')
print("Input batch:")
print(x)
print("Target batch:")
print(y)    

Input batch:
tensor([[65, 64, 72,  1, 57, 68, 67,  5],
        [74,  1, 72, 74, 71, 69, 71, 62],
        [74, 62, 56, 64,  2,  1, 44, 68],
        [67, 57,  1, 73, 68, 65, 57,  1]], device='cuda:0')
Target batch:
tensor([[64, 72,  1, 57, 68, 67,  5, 73],
        [ 1, 72, 74, 71, 69, 71, 62, 72],
        [62, 56, 64,  2,  1, 44, 68,  1],
        [57,  1, 73, 68, 65, 57,  1, 74]], device='cuda:0')


In [55]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [52]:
# basic bigram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape  # B=batch size, T=time, C=channels (vocab size)
            
            # reshape logits and targets for cross-entropy loss
            logits = logits.view(B*T, C) 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)  # get predictions
            logits = logits[:, -1, :] # (B, C) last time step
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size).to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)        


D'4 NdFYCjBae31qSYeQGklP]Tb*b
m
-w_?tG-:"hY*qdO* 3;KW,J)G)mRu'OQOCxt&H,tNdtFg"l6s"&)8,eGW:8NGcn-uJtXJ6GhsUJ[V;llZxCgGe;t6!uhkFJFfqb*VXGLA")OY(Z'0CP7 3rN3h64E3)3aNMb,vTPFgoX5)rAl&?yyeKWpv(0 )3y_&2MK"oE:NxA[_? 'BS5T?lDvp.PAft.1je 09s.,vdAzkhw
?KOE27Ir1C!at&kK,qz NW*GN7J[Mb[qX![VvlP(_!q7ajU4a3-LsUVUf:y[12"_vY)N423vEm0l?2uUC8;5GB.e9H
2iyGP3QrBE F5GWm mJ0&,-pLE
9HSMTXQ_[7N5o;A[ZO*2MIgwKv4k[UmYWf9rF6(p__jA[O0jB '_Zn4 
[EY![ tZ!ZlJ"r9Usm2M73wPQ.6':
9PMKP(ZOcDwRp)hiM 0GAI
?teZkTXIf'B359HTnpOQjXYUOIdUM)c


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.8690, val loss: 2.9126
step: 250, train loss: 2.8647, val loss: 2.8847
step: 500, train loss: 2.8575, val loss: 2.8516
step: 750, train loss: 2.8208, val loss: 2.8252
2.8510630130767822


In [66]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # initial context
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars) 


w8LJNqorett.

IU"h3
Iit  ctigg ZbAH2O:QUN*On!an "F[2ste otug-Ln ky"W1

rtin T"KMZhe;2utced S[sckn&:S0lfomauCYEY athaped idin'lled mpthe. t02.

vY84E?AIle, acR123s ceis
kTh)myNE9Q.4
Ltird U_lio,P3hisiry
Z!
SZ,mQqI7Zn'il A9ngls,"YHv2MrkBGW*FF"r;LIBL[U_SoVJT2MkT0BVysn,T)4xXNT316PBF'L(xCHk r?H(t.Pt
vPVAnl!ofd h)"MKmalsedamp?oxOY(eGe V&
jat bD.Ls!(Xf1odoy,'towr  acrttheliliob, ilsthmioe suyl.qO*rsz, 2N)h6TyYavin-?he sm2BEulyhed., pelja?:P1ofvrthelle ou oblie d DD0ThklL'l;
blorto.
im.yf':EEIg."eritrAs
